In [1]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2023-12-30 01:28:58--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: 'input.txt.1'

     0K .......... .......... .......... .......... ..........  4% 3.10M 0s
    50K .......... .......... .......... .......... ..........  9% 9.05M 0s
   100K .......... .......... .......... .......... .......... 13% 4.82M 0s
   150K .......... .......... .......... .......... .......... 18% 11.7M 0s
   200K .......... .......... .......... .......... .......... 22% 8.70M 0s
   250K .......... .......... .......... .......... .......... 27% 18.7M 0s
   300K .......... .......... .......... .......... .......... 32% 16.5M 0s
   350K .......... 

In [2]:
with open('input.txt', 'r',encoding='utf-8') as f:
    text = f.read()

In [3]:
print("length of dataset in characters:", len(text))

length of dataset in characters: 1115394


In [4]:
print(text[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [5]:
chars = sorted(list(set(text)))
vocab_Size = len(chars)
print(vocab_Size, ''.join(chars))

65 
 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz


#### tokenization

In [6]:
stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

print(encode("hii there"))
print(decode(encode("hii there")))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [24]:
# import tiktoken
# enc = tiktoken.get_encoding('gpt2')
# print(enc.n_vocab)
# print(enc.encode("hii there"))
# print(enc.decode(enc.encode("hii there")))

50257
[71, 4178, 612]
hii there


In [25]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:100])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [26]:
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [45]:
block_size = 8
train_data[:block_size+1]#.tolist()

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [46]:
decode(train_data[:block_size+1].tolist())

'First Cit'

In [47]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target is: {target}")

when input is tensor([18]) the target is: 47
when input is tensor([18, 47]) the target is: 56
when input is tensor([18, 47, 56]) the target is: 57
when input is tensor([18, 47, 56, 57]) the target is: 58
when input is tensor([18, 47, 56, 57, 58]) the target is: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target is: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target is: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target is: 58


In [48]:
batch_size = 4
block_size = 8

def get_batch(split):
    data = train_data if split=='train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i : i + block_size] for i in ix])
    y = torch.stack([data[i + 1 : i + block_size + 1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print("inputs:")
print(xb.shape)
print(xb)

print('tragets:')
print(yb.shape)
print(yb)

print('--------')

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b, t]
        print(f"when input is {context} the target is: {target}")

inputs:
torch.Size([4, 8])
tensor([[ 0, 35, 46, 43, 56, 43,  1, 44],
        [10,  0, 32, 46, 39, 58,  5, 57],
        [56, 44, 59, 50,  1, 58, 47, 51],
        [42,  1, 58, 53,  1, 39, 45, 39]])
tragets:
torch.Size([4, 8])
tensor([[35, 46, 43, 56, 43,  1, 44, 43],
        [ 0, 32, 46, 39, 58,  5, 57,  1],
        [44, 59, 50,  1, 58, 47, 51, 43],
        [ 1, 58, 53,  1, 39, 45, 39, 47]])
--------
when input is tensor([0]) the target is: 35
when input is tensor([ 0, 35]) the target is: 46
when input is tensor([ 0, 35, 46]) the target is: 43
when input is tensor([ 0, 35, 46, 43]) the target is: 56
when input is tensor([ 0, 35, 46, 43, 56]) the target is: 43
when input is tensor([ 0, 35, 46, 43, 56, 43]) the target is: 1
when input is tensor([ 0, 35, 46, 43, 56, 43,  1]) the target is: 44
when input is tensor([ 0, 35, 46, 43, 56, 43,  1, 44]) the target is: 43
when input is tensor([10]) the target is: 0
when input is tensor([10,  0]) the target is: 32
when input is tensor([10,  0, 32]) 